In [1]:
!pip install vaderSentiment

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import time
import os
import json
from sklearn.metrics import accuracy_score , precision_score, recall_score , f1_score
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
test_data = pd.read_csv("/content/drive/MyDrive/Sentiment-Analysis/Data/wo. Neutral Splits/TestFile.csv", header=None)
test_data.columns = ['review', 'sentiment']

In [5]:
# test_data = test_data.drop(0)

In [6]:
analyzer = SentimentIntensityAnalyzer()

def calculate_sentiment_scores(sentence):
    sntmnt = analyzer.polarity_scores(sentence)['compound']
    return(sntmnt)

start = time.time()

eng_snt_score =  []

for review in test_data['review']:
    snts_score = calculate_sentiment_scores(review)
    eng_snt_score.append(snts_score)

end = time.time()

test_data['sentiment_score'] = eng_snt_score
vader_sentiment = [ ]
for i in range(len(test_data)):
    sentiment_score = test_data.iloc[i]['sentiment_score']

    if sentiment_score > 0:
        vader_sentiment.append('positive')
    elif sentiment_score <= 0:
        vader_sentiment.append('negative')

test_data['vader_sentiment_labels'] = vader_sentiment


test_data['actual_label'] = test_data.iloc[:, 1].map({'positive': 0, 'negative':1})
test_data['predicted_label'] = test_data['vader_sentiment_labels'].map({'positive': 0, 'negative':1})

In [7]:
with open("/content/drive/MyDrive/Sentiment-Analysis/Data/evaluation.json", 'r') as json_file:
          df = json.loads(json_file.read())

In [8]:
for i, key in enumerate(df.keys()):
        df[key]['lexicon'] = int(test_data['predicted_label'][i])
        df[key]['actual'] = int(test_data['actual_label'][i])

In [9]:
with open("/content/drive/MyDrive/Sentiment-Analysis/Data/evaluation.json", "w") as json_file:
    json.dump(df, json_file)

In [10]:
y_act = test_data['actual_label'].values
y_pred = test_data['predicted_label'].values
accuracy = accuracy_score(y_act, y_pred)
print(f'acc:{accuracy}')
precision = precision_score(y_act, y_pred, average='macro')
print(f'precision: {precision}')
recall = recall_score(y_act, y_pred, average='macro')
print(f'recall:{recall}')
f1 = f1_score(y_act, y_pred, average='macro')
print(f'f1:{f1}')



acc:0.836574892649659
precision: 0.6783556828830002
recall:0.7819630118167387
f1:0.7073462640873271


In [11]:
precision_w = precision_score(y_act, y_pred, average='weighted')
print(f'precision: {precision_w}')
recall_w = recall_score(y_act, y_pred, average='weighted')
print(f'recall:{recall_w}')
f1_w = f1_score(y_act, y_pred, average='weighted')
print(f1_w)

precision: 0.88813496755019
recall:0.836574892649659
0.854681047616998
